<h1><center>  实体标记 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取</center></h1>


 


## 目标:
-	在给定a-prioris列表的情况下，使用在给定数据集的每个文档中找到的实体生成元数据


## 数据:
-	任何文件集合

-   在你感兴趣的数据集中检测和标记的实体列表


## Nucleus APIs used:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	数据集标记 API
 - 	*api_instance.post_dataset_tagging(payload)*


-	文件信息 API
 - 	*api_instance.post_doc_info(payload)*


## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
print('---- Upload documents from a local folder into a new Nucleus dataset ----')
folder = 'Corporate_documents'         
dataset = 'Corporate_docs'# str | 文件的目标数据集。

# 以递归方式从文件夹构建文件。 
# 每项都采用以下格式
# {'filename': filename,   # 要上传的文件名。 必填
#  'metadata': {           # 该文件的元数据。 可选
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
        file_dict = {'filename': os.path.join(root, file),
                     'metadata': {'category': 'News'}} # 没有每个新闻的代码，让我们标记它们
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

### 2.	数据集标记
-	定义公司代码（或与您相关的任何其他实体）的列表


-	遍历此列表并使用数据集标记API来确定哪些文档包含给定的代码


-	接下来，我们将讨论如何构建自定义的停用词列表以优化文档摘要



In [ ]:
print('---------------- Tag dataset ------------------------')

payload = nucleus_api.DatasetTaggingModel(dataset='Corporate_docs', 
                                    query='AAPL OR Apple', 
                                    metadata_selection='', 
                                    time_period='')
try:
    api_response = api_instance.post_dataset_tagging(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    print('Information about dataset', dataset)
    print('    Entity Tagged:', api_response.result.entities_tagged)
    print('    Docids tagged with Entity:', api_response.result.doc_ids)
    print('    Entity occurrences in each docid:', api_response.result.entities_count)

现在我们创建实体列表并遍历它

In [ ]:
entities = [['AAPL', 'Apple'], ['GOOG', 'Google', 'Alphabet']]

docs_tagged = []
entities_tagged = []
for i in range(len(entities)):
    query = " OR ".join(entities[i])
    payload = nucleus_api.DatasetTaggingModel(dataset='Corporate_docs', 
                                    query=query, 
                                    metadata_selection='', 
                                    time_period='')
    try:
        api_response = api_instance.post_dataset_tagging(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:
        for docid in api_response['doc_ids']::
            docs_tagged.append(docid)
            entities_tagged.append(api_response['entities_tagged'][0]) # 保留实体的第一个命名作为标签

        # 让我们重新组合每个文档标记的实体，这样我们就有了一个唯一的docid列表以及标记在其中的所有实体

        # 此表对于生成带有作为元数据提供的代码的更新数据集非常有用，因为我们真正关心的是文件名而不是docids
        from collections import defaultdict
        d = defaultdict(list)
        for i, entity in enumerate(entities_tagged):
            payload = nucleus_api.DocInfo(
                dataset='Corporate_docs', 
                doc_ids=docs_tagged[i],
                metadata_selection='')
            api_response = api_instance.post_doc_info(payload)
            key = api_response.result[0].attribute['filename']
            d[key].append(entity)
        d = dict(d)

使用这些标签，我们可以构建第二个额外元数据的数据集，这在信号研究和合规性分析中非常方便。

我们可以使用文件名将原始文档与已标记的文档进行匹配

In [ ]:
dataset = 'Corporate_docs_2'# str | 文件的目标数据集。

file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
        
        # 我们知道当前正在注入的文件的文件名，我们可以将它与标记文档表进行匹配
        if d[os.path.join(root, file)] != [] # 仅使用具有标记实体的文档构建新数据集
            tickers = d[os.path.join(root, file)]
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'companies': tickers,
                                      'category': 'News'}}
            
            file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

### 3.	微调

#### a.	扩展给定实体的同义词列表
**query**: 您可以优化数据集标记并扩展您的代码列表（或其他相关实体）以包含任意数量的备选方案。 

您还可以创建一次保守的超集列表，保留该列表并将其重新用于您要标记的每个数据集。

上述也适用与外国公司。 例如，您可以将列表的条目定义为['Nintendo'，'NTDOY'，'任天堂株式会社']

将该扩展列表（循环遍历所有不同的代码）传递给第2节主代码中的查询参数，然后重新运行该代码：


In [ ]:
entities = [['AAPL', 'Apple', 'iPhone'], ['GOOG', 'Google', 'Alphabet', 'Android'], ['NTDOY', 'Nintendo', '任天堂株式会社']]

docs_tagged = []
entities_tagged = []
for i in range(len(entities)):
    query = " OR ".join(entities[i])
    payload = nucleus_api.DatasetTaggingModel(dataset='Corporate_docs', 
                                    query=query, 
                                    metadata_selection='', 
                                    time_period='')
    try:
        api_response = api_instance.post_dataset_tagging(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:
        for docid in api_response['doc_ids']:
            docs_tagged.append(docid)
            entities_tagged.append(api_response['entities_tagged'][0]) # 保留实体的第一个命名作为标签
            
        # 我们重新组合每个文档标记的实体，这样我们就有了一个唯一的docid列表以及标记在其中的所有实体
        # 此表对于生成更新的数据集非常有用，因此我们真正关心的是文件名而不是docids
        d = defaultdict(list)
        for i, entity in enumerate(entities_tagged):
            payload = nucleus_api.DocInfo(
                dataset='Corporate_docs', 
                doc_ids=docs_tagged[i],
                metadata_selection='')
            api_response = api_instance.post_doc_info(payload)
            key = api_response.result[0].attribute['filename']
            d[key].append(entity)
        d = dict(d)

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 